# Ingestao_Azure_Storage_DataFactory

## Problemas na Injestão de dados

No Data Factory Azure para realizarmos a injestão dos dados para uma Conteiner na conta de armazenamento utilizando a conexão com a API do MuseusBr em um Copia dados em seu “Source” como temos na figura abaixo:

<img src="01_CopiaDados.png">

Em um Data Set, como temos na figura abaixo:

<img src="02_Servico_Vinculado.png">

Ao executarmos esta atividade para copiar os dados para o Conteiner “lindberg-ferreira / lab04 / Bronze”, obtemos a seguinte inconsistência:

<img src="imagen\03_MensagemErro.png">

<img src="imagen\04_ErroCopiaData.png">

Quando exploramos o erro vemos uma inconsistência com permissão de acesso e o Azure Integrate rumtime. Em pesquisas conseguimos chegar na informação neste <a src="https://docs.microsoft.com/pt-br/azure/data-factory/create-self-hosted-integration-runtime?tabs=data-factory">link</a> :

<i><b>Um runtime de integração auto-hospedada pode executar atividades de cópia entre um armazenamento de dados de nuvem e um armazenamento de dados em uma rede privada. Ele também pode distribuir atividades de transformação em relação aos recursos de computação em uma rede local ou em uma rede virtual do Azure. A instalação de um runtime de integração auto-hospedada precisa de um computador local ou uma máquina virtual em uma rede privada.</b></i>

Este é o nosso caso, onde temos uma rede do governo em servidor próprio (privado) no Brasil e estamos acessando de um ponto de acesso em outra localiade no Azure.

## Integration Runtime

O IR (runtime de integração) é a infraestrutura de computação que os pipelines do Azure Data Factory e do Synapse usam para fornecer funcionalidades de integração de dados entre diferentes ambientes de rede:

 <ul>
    <li>Fluxo de Dados: execute um fluxo de dados no ambiente de computação gerenciado do Azure.<br><br>
    <li>Movimentação de dados: copiar dados entre armazenamentos de dados em redes públicas ou privadas (para redes locais ou privadas). O serviço fornece suporte para conectores internos, conversão de formato, mapeamento de coluna e transferência de dados de alto desempenho e escalonáveis.<br><br>
    <li>Expedição de atividades: distribua e monitore atividades de transformação em execução em uma variedade de serviços de computação, como Azure Databricks, Azure HDInsight, Estúdio ML (clássico), Banco de Dados SQL do Azure, SQL Server e muito mais.<br><br>
    <li>Execução de pacote SSIS: execute nativamente pacotes do SSIS (SQL Server Integration Services) em um ambiente de computação gerenciada do Azure.
</ul>

<p>Nos pipelines do Data Factory e do Synapse, uma atividade define a ação a ser executada. Um serviço vinculado define um armazenamento de dados de destino ou um serviço de computação. Um Integration Runtime fornece a ponte entre as atividades e os serviços vinculados. Ele é referenciado pelo serviço vinculado ou pela atividade e fornece o ambiente de computação no qual a atividade é executada diretamente ou de onde é expedida. Isso permite que a atividade seja executada na região mais próxima possível do armazenamento de dados de destino ou do serviço de computação a fim de maximizar o desempenho e, ao mesmo tempo, permitindo flexibilidade para atender aos requisitos de segurança e conformidade.</p>
<p>Os runtimes de integração podem ser criados diretamente no Azure Data Factory e na interface do usuário do Azure Synapse por meio do hub de gerenciamento, bem como de atividades, conjuntos de dados ou fluxos de dados que os referenciem.<p>

<img src="imagen\05_hubGerenciamento.png">

<p>O Data Factory oferece três tipos de Integration Runtime (IR) e você deve escolher o tipo que melhor atende aos seus requisitos de funcionalidades de integração de dados e de ambiente de rede. Os três tipos de IR são:</p>
<ul>
    <li>Azure
    <li>Auto-hospedado
    <li>Azure-SSIS
</ul>
<p>A tabela a seguir descreve as funcionalidades e o suporte de rede para cada um dos tipos de Integration Runtime:</p>

<img src="imagen\05_1_tiposIR.png">

Um IR <b><a src="https://docs.microsoft.com/pt-br/azure/data-factory/concepts-integration-runtime#self-hosted-integration-runtime">auto-hospedado</a></b> é capaz de:
<ul>
    <li>Executar a atividade de cópia entre um armazenamento de dados de nuvem e um armazenamento de dados na rede privada.<br><br>
    <li>Expedir as seguintes atividades de transformação para recursos de computação na rede virtual local ou do Azure: atividades de Hive do HDInsight (BYOC, Traga Seu Próprio Cluster), de Pig do HDInsight (BYOC), de MapReduce do HDInsight (BYOC), de Spark do HDInsight (BYOC), de Streaming do HDInsight (BYOC), de Batch Execution (clássico) do Estúdio do ML, de Recurso de Atualização do Estúdio do ML (clássico), de Procedimento Armazenado, de U-SQL do Data Lake Analytics, atividade personalizada do .Net (executado no Lote do Azure), atividade de pesquisa e atividade de obtenção de metadados.
</ul>
Observação
<ul>
    <li>Use o runtime de integração auto-hospedado para dar suporte a armazenamentos de dados que requer traga seu próprio driver como SAP Hana, MySQL, etc. Para mais informações, confira os armazenamentos de dados suportados.<br><br>
    <li>O JRE (Java Runtime Environment) é uma dependência do IR auto-hospedado. Verifique se o JRE está instalado no mesmo host.
</ul>
<p>Se você deseja realizar a integração de dados com segurança em um ambiente de rede privada, que não tem uma linha de visão direta do ambiente de nuvem pública, pode instalar um IR auto-hospedado no seu ambiente local por trás de seu firewall ou então em uma rede virtual privada. O runtime de integração auto-hospedado só faz conexões de saída com baseadas em HTTP com a Internet.</p>
<p>Instale um IR auto-hospedado em um computador local ou em uma máquina virtual dentro de uma rede privada. Atualmente, o IR auto-hospedado só tem suporte em um sistema operacional Windows.</p>
<p>Para alta disponibilidade e escalabilidade, você pode expandir o IR auto-hospedado associando a instância lógica a vários computadores locais no modo ativo-ativo. Para obter mais informações e detalhes, confira o artigo em <b><a src="https://docs.microsoft.com/pt-br/azure/data-factory/create-self-hosted-integration-runtime?tabs=data-factory">Como criar e configurar um IR auto-hospedado</a></b>.</p>

## Instalação Integration Runtime auto hospedado (Self)

<p>O IR (runtime de integração) é a infraestrutura de computação que os pipelines do Azure Data Factory e do Synapse usam para fornecer funcionalidades de integração de dados entre diferentes ambientes de rede. Para obter detalhes sobre o IR, confira Visão geral do Integration Runtime.</p>
<p>Um runtime de integração auto-hospedada pode executar atividades de cópia entre um armazenamento de dados de nuvem e um armazenamento de dados em uma rede privada. Ele também pode distribuir atividades de transformação em relação aos recursos de computação em uma rede local ou em uma rede virtual do Azure. A instalação de um runtime de integração auto-hospedada precisa de um computador local ou uma máquina virtual em uma rede privada.</p>
<b><a src="https://docs.microsoft.com/pt-br/azure/data-factory/create-self-hosted-integration-runtime?tabs=data-factory#considerations-for-using-a-self-hosted-ir">Considerações para</a></b>Considerações para o uso de um IR auto-hospedado:
<ul>
    <li>Você pode usar um único runtime de integração auto-hospedada para várias fontes de dados locais. Você também pode compartilhá-lo com outro data factory no mesmo locatário do Azure AD (Azure Active Directory). Para obter mais informações, confira Compartilhando um runtime de integração auto-hospedada.<br><br>
    <li>Você pode instalar apenas uma instância do runtime de integração auto-hospedada em um único computador. Se você tiver dois workspaces do data factories ou do Synapse que precisam acessar fontes de dados locais, use o recurso de compartilhamento de IR auto-hospedado para compartilhar o IR auto-hospedado ou instale o IR auto-hospedado em dois computadores locais, um para cada workspace do data factory do Synapse.<br><br>
    <li>O runtime de integração auto-hospedada não precisa estar no mesmo computador que a fonte de dados. No entanto, ter o runtime de integração auto-hospedada mais perto da fonte de dados reduz o tempo para o runtime de integração auto-hospedada se conectar à fonte de dados. É recomendável instalar o runtime de integração auto-hospedada em um computador que seja diferente daquele que hospeda a fonte de dados local. Quando a fonte de dados e o runtime de integração auto-hospedada estão em computadores diferentes, o runtime de integração auto-hospedada não compete por recursos com a fonte de dados.<br><br>
    <li>Você pode ter vários runtimes da integração auto-hospedada em diferentes computadores conectados à mesma fonte de dados local. Por exemplo, se você tiver dois runtimes de integração auto-hospedada servindo dois data factories, a mesma fonte de dados local pode ser registrada com ambos os data factories.<br><br>
    <li>Use um runtime de integração auto-hospedada para dar suporte à integração de dados na rede virtual do Azure.<br><br>
    <li>Trate a fonte de dados como uma fonte de dados local protegida por um firewall mesmo quando você usar o Microsoft Azure ExpressRoute. Use o runtime de integração auto-hospedada para conectar o serviço ao data factory.
    <li>Use o runtime de integração auto-hospedada mesmo se o repositório de dados estiver na nuvem em uma máquina virtual de IaaS (infraestrutura como serviço) do Azure.<br><br>
    <li>As tarefas podem falhar em um runtime de integração auto-hospedada instalado em um Windows Server para o qual a criptografia em conformidade com FIPS está habilitada. Para contornar esse problema, você tem duas opções: armazenar credenciais/valores de segredo em um Azure Key Vault ou desabilitar a criptografia compatível com FIPS no servidor. Para desabilitar a criptografia compatível com FIPS, altere o valor da subchave de registro a seguir de 1 (habilitado) para 0 (desabilitado): HKLM\System\CurrentControlSet\Control\Lsa\FIPSAlgorithmPolicy\Enabled. Se você usar o runtime de integração auto-hospedada como proxy para o runtime de integração do SSIS, a criptografia compatível com FIPS poderá ser habilitada e será usada ao migrar dados do local para o Armazenamento de Blobs do Azure como área de preparo.<br><br>
</ul>
<p>Quando você move os dados entre a nuvem e o local, a atividade usa um runtime de integração auto-hospedada para transferir os dados entre uma fonte de dados local e a nuvem.</p>
<p>Este é o resumo de alto nível das etapas do fluxo de dados para a cópia com um IR auto-hospedado:</p>

<img src="imagen\06_fluxoIR.png">

<ol>
    <li>Um desenvolvedor de dados cria um runtime de integração auto-hospedado em um workspace do Azure Data Factory ou do Synapse usando o portal do Azure ou o cmdlet do PowerShell. Em seguida, ele cria um serviço vinculado para um armazenamento de dados local especificando a instância de runtime de integração auto-hospedado que o serviço deve usar para se conectar aos armazenamentos de dados.<br><br>
    <li>O nó do runtime de integração auto-hospedada criptografa a credencial usando a DPAPI (Interface de Programação do Aplicativo de Proteção de Dados) do Windows e salva as credenciais localmente. Se vários nós forem definidos para alta disponibilidade, as credenciais serão mais sincronizadas em outros nós. Cada nó criptografa as credenciais usando a DPAPI e as armazena localmente. A sincronização de credenciais é transparente para o desenvolvedor de dados e é tratada pelo IR auto-hospedado.<br><br>
    <li>Os pipelines do Azure Data Factory e do Synapse se comunica com o runtime de integração auto-hospedada para agendar e gerenciar trabalhos. A comunicação ocorre por meio de um canal de controle que usa uma conexão de Retransmissão do Azure compartilhada. Quando o trabalho de atividade precisa ser executado, o serviço enfileira a solicitação juntamente com as informações de credencial. Ele faz isso, caso as credenciais ainda não estejam armazenadas no runtime de integração auto-hospedada. O runtime de integração auto-hospedada inicia o trabalho depois de sondar a fila.<br><br>
    <li>O runtime de integração auto-hospedada copia dados entre um armazenamento local e um armazenamento em nuvem. A direção da cópia depende de como a atividade Copy foi configurada no pipeline de dados. Para esta etapa, o runtime de integração auto-hospedada se comunica diretamente com os serviços de armazenamento baseado em nuvem, como Armazenamento de Blobs do Azure por um canal HTTPS seguro.
</ol>

## Configurar um IR auto-hospedado na interface do usuário

Use as etapas a seguir para criar um IR auto-hospedado usando a interface do usuário do Azure Data Factory:

<ol>
<li>Na página inicial da interface do usuário do Azure Data Factory, selecione a guia Gerenciar no painel mais à esquerda.     <br><img src="imagen\07_guiaGerenciar.png"></li><br><br>
    
<li>Selecione Runtimes de integração no painel esquerdo e, em seguida, selecione <b>+ Novo. </b> <br><img src="imagen\08_novoIR.png"><br><br>
<li>Na página Configuração do runtime de integração, selecione Azure, Auto-Hospedado e Continuar.
        <br><img src="imagen\09_IRSelf.png"><br><br>
<li>Na página a seguir, selecione Auto-Hospedado para criar um IR auto-hospedado e, em seguida, selecione Continuar.
        <br><img src="imagen\10_IR_Self.png"><br><br>
<li>Escolher nome e descrição, após criar.
        <br><img src="imagen\11_criarIRSelf.png"><br><br>
<li>Na página Configuração do runtime de integração, selecione o link em Opção 1 para abrir a configuração expressa no computador. Ou siga as etapas em Opção 2 para configurar manualmente. As instruções a seguir são baseadas na configuração manual:
        <br><img src="imagen\12_instalacaoIRManual.png"><br><br>
<li>Copie e cole a chave de autenticação <b>Key 1</b>. Selecione Baixar e instalar o runtime de integração. Baixe o runtime de integração auto-hospedada em uma máquina local do Windows. Execute o instalador.
        <br><img src="imagen\13_baixarIR.png"><br><br>
<li>Para instalação, apenas clicar em proximo, e seguir.
        <br><img src="imagen\14_instaladorIR.png"><br><br>
<li>Na página Registrar Integration Runtime (auto-hospedado) , cole a chave salva anteriormente, a <b>Key1</b>, e selecione Registrar, após, Na página Novo nó do Integration Runtime (auto-hospedado) , selecione Concluir.
        <br><img src="imagen\15_colarChave.png"><br><br>
<li>Depois que o runtime de integração auto-hospedada estiver registrado com sucesso, você verá a seguinte janela:
        <br><img src="imagen\16_IR_Instalado.png"><br><br>
<li>Se tudo estiver configurado com sucesso no Gerenciar -> Integration runtime estara:
        <br><img src="imagen\17_IR_Running.png">    
     
</ol>

## Copia dados Data Factory

<p>No copia dados no Data Factory, em pipelines, vamos voltar no nosso Data Set (conjunto de dados), e editar o nosso Linked Service (serviço vinculado) para aponatar, utilizar nosso novo Integration runtime:</p>
<img src="imagen\18_ConfigurandoLinkServIRNov.png">

<p>Vamos clicar em “Publish all” para não perdemos o que fizemos, e testar nosso pipeline:</p>
<img src="imagen\19_publishTestPipeline.png">

<p>Obsever que o horario dos “Time Zone” são diferentes, por conta disso também utilizamos o integartion runtime para realizar o sincornismos, e garantir a comunicação entre os servidores da API e do Azure:</p>
<img src="imagen\20_IngestaoTrue.png">